<a href="https://colab.research.google.com/github/jeawonlll/KOHI2022_tutorials/blob/main/20220903/%5Bpublic%5D_07_lowbp_preprocessing_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 의료인공지능 전문가 양성과정 2022
## VitalDB Tutorial <br> Low blood pressure prediction using arterial wave - preprocessing
- Date : Sep. 03, 2022
- Author : **Hyun-Lim Yang, Ph.D.**<br>
Research Assistant Professor @
Seoul National University Hospital <br>
Department of Anesthesiology and Pain Medicine
- E-mail : hlyang{_at_}snu{_dot_}ac{_dot_}kr
***

In [ ]:
from IPython.display import HTML
style_warn = "<style>div.warn { background-color: #fcf2f2;border-color: #dFb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;}</style>"
HTML(style_warn)

### Import packages

<div class="warn">**Warning!** : use your directory at `download_directory`</div>


> **Wargning** <br>
> colab 환경을 위한 google drive import 코드가 포함되어 있습니다. <br>
> 로컬 환경에서 실행 시 colab을 위한 import function들을 comment out 한 뒤 실행하세요. 

In [ ]:
!pip install vitaldb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 353 kB/s 
     |████████████████████████████████| 161 kB 49.4 MB/s 
     |████████████████████████████████| 65 kB 2.3 MB/s 
     |████████████████████████████████| 140 kB 52.3 MB/s 
     |████████████████████████████████| 9.1 MB 39.1 MB/s 
     |████████████████████████████████| 127 kB 58.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1


In [ ]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab

Mounted at /content/gdrive/


In [ ]:
import os
cloud_directory = '/content/gdrive/MyDrive/KOHI_2022_CNN_data_open/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

['sample_csv',
 'datasets',
 '__init__.py',
 '__pycache__',
 'kohi_CNN_model_archi.png',
 'kohi_preprocessor.py']

In [ ]:
import sys
#download_directory = os.getcwd() # for local environments
#sys.path.append(download_directory) # for local environments
sys.path.append(cloud_directory) # for colab

In [ ]:
import os
import sys
# download_directory = os.getcwd() # for local environments
# sys.path.append(download_directory) # for local environments

In [ ]:
import numpy as np
import pandas as pd
import glob
import kohi_preprocessor as pre
import vitaldb
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

download_directory = cloud_directory # for colab

### Data loading
샘플 파일을 vitaldb 서버로 부터 직접 다운받아 로드

> **TODO:** `00010.vital`을 100hz로 변환하여 `chart_pd_01`에 DataFrame으로 저장하기

In [ ]:
track_names = ["SNUADC/ART", "Solar8000/ART_MBP"]
### =========== Your code here ====================

vitalfile = vitaldb.VitalFile(10, track_names=track_names)
chart_pd_01 = vitalfile.to_pandas(track_names, interval=1/100)

### ===============================================


In [ ]:
chart_pd_01

,SNUADC/ART,Solar8000/ART_MBP
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
2099215,NaN,NaN
2099216,NaN,NaN
2099217,NaN,NaN
2099218,NaN,NaN


In [ ]:
col_mbp = 'Solar8000/ART_MBP'
col_art = 'SNUADC/ART'

### mean blood pressure 데이터 추출
> **TODO:** ART_MBP 변수들 중 값이 존재하는 것만 별도의 변수 `mbp_data_pd`에 저장

In [ ]:
# mbp 데이터 추출
### =========== Your code here ====================

mbp_data_pd = chart_pd_01[col_mbp][chart_pd_01[col_mbp].notnull()]
#대략 200초마다 계산되므로 

### ===============================================

mbp_index = mbp_data_pd.index.values
print(mbp_data_pd.head(1000))
#데이터 제대로 connection 안되어있어서 뒤에서 처리해야함.

708      -20.0
899      -20.0
1099     -20.0
1299     -20.0
1499     -20.0
          ... 
199705    54.0
199905    54.0
200105    54.0
200305    54.0
200505    54.0
Name: Solar8000/ART_MBP, Length: 1000, dtype: float32


### arterial wave 데이터 추출 및 nan 지우기
> **TODO:** ART 변수들 중 값이 존재하지 않는 모든 부분을 0으로 채운 뒤 `art_full_pd`에 저장

In [ ]:
# art 데이터 전체 추출 및 nan value 채우기
art_full_pd = chart_pd_01[col_art]

### =========== Your code here ====================

art_full_pd = art_full_pd.fillna(0)

### =========== Your code here ====================
print(art_full_pd.head(1000))

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
         ...    
995   -17.696800
996   -18.684299
997   -18.684299
998   -17.696800
999   -18.684299
Name: SNUADC/ART, Length: 1000, dtype: float32


### 데이터셋 만들기
필요한 파라미터들 정의

In [ ]:
# 필요한 파라미터들 정의
mdelay = 1 #1분뒤에 저혈압 예측
srate = 100 #100hz
length = 20 #20의 a라인 보고 bp볼거기때문에
max_limit_mbp = 250 #단순하게 min,max 설정
min_limit_mbp = 0

입력 데이터 길이가 20s이고 black out (여기서는 mdelay) 기간이 1분 이니, mbp가 1분 20초 이내에 등장하는 것은 무시함

In [ ]:
# mbp index를 1분 20초 뒤로 미룸
mbp_points = mbp_index[mbp_index > (mdelay*60*srate + length*srate)]
print(mbp_points)

[   8100    8300    8500 ... 2092959 2093159 2093359]


### Arterial wave segment 추출
> **TODO:** 현재 mbp 값을 기준으로, 1분 20초 전 ~ 1분 전 데이터 (즉, 입력 데이터 20s)를 arterial wave segment로 추출하여 `art_seg_list`에 저장하기

In [ ]:
# 실습: arterial segment 추출하기
mbp_values_list = [mbp_data_pd[idx] for idx in mbp_points]

### =========== Your code here ====================

#1:20초 전부터 1분까지
art_seg_list = [art_full_pd[idx-(mdelay*60*srate) - (length*srate) : 
                            idx - (mdelay*60*srate)].values 
                for idx in mbp_points]
### ===============================================

In [ ]:
mbp_values_np = np.array(mbp_values_list)
art_seg_np = np.array(art_seg_list)
print(mbp_values_np.shape)
print(art_seg_np.shape)

(10428,)
(10428, 2000)


### 조건에 따라 filter들 정의

In [ ]:
# 필터들 선언
# mbp min-max filter
mbp_max_filter = mbp_values_np < max_limit_mbp
mbp_min_filter = mbp_values_np > min_limit_mbp
mbp_filter = mbp_max_filter & mbp_min_filter

# abp range filter
art_filter_list = []
for seg in art_seg_np:
    filter_value = (np.array(seg) > 25.0).all() and (np.array(seg) < 250.0).all()
    art_filter_list.append(filter_value)
art_filter = np.array(art_filter_list)

# mstds 필터
mstds_values_list = []
for seg in tqdm(art_seg_np):
    if (np.array(seg) < 0.).any():
        mstds_values_list.append(float(0.))
    else:
        mstd_val, _ = pre.process_beat(seg)
        mstds_values_list.append(mstd_val)
mstds_filter = np.array(mstds_values_list) > 0.

100%|██████████| 10428/10428 [02:03<00:00, 84.28it/s] 


전체 필터 하나로 만들기

In [ ]:
all_filters = mbp_filter & art_filter & mstds_filter

### 필터 적용하여 데이터 추출
> **TODO:** `all_filters`를 이용하여 필터 통과한 데이터들만 추출해 `mbp_filtered`와 `art_filtered`에 저장

In [ ]:
### =========== Your code here ====================

mbp_filtered = mbp_values_np[all_filters]
art_filtered = art_seg_np[all_filters]

### ===============================================

print(mbp_filtered.shape)
print(art_filtered.shape)

(8110,)
(8110, 2000)


데이터셋 정의

In [ ]:
mbp_filtered[mbp_filtered < 60].shape

(640,)

### Binary label 정의
mbp가 61 미만인 데이터들의 label을 1로 지정하여 학습을 위한 y_label 생성

In [ ]:
label_tmp = np.zeros(len(mbp_filtered))
label_tmp[mbp_filtered<61] = 1.

In [ ]:
x_data = art_filtered
y_label = label_tmp

In [ ]:
x_data.shape

(8110, 2000)

In [ ]:
y_label.shape

(8110,)